# Multi-objective Robust Optimization (MORO)


This exercise demostrates the application of MORO on the lake model. In contrast to the exercises in previous weeks, we will be using a slightly more sophisticated version of the problem. For details see the MORDM assignment for this week.

## Setup MORO

Many objective robust optimization aims at finding decisions that are robust with respect to the various deeply uncertain factors. For this, MORO evalues each candidate decision over a set of scenarios. For each outcome of interest, the robusntess over this set is calculated. A MOEA is used to maximize the robustness. 

For this assignment, we will be using a domain criterion as our robustness metric. The table below lists the rules that you should use for each outcome of interest.

|Outcome of interest| threhsold  |
|-------------------|------------|
| Maximum pollution | $\leq$ 0.75|
| Inertia           | $\geq$ 0.6 |
| Reliability       | $\geq$ 0.99|   
| Utility           | $\geq$ 0.75|

**1) Implement a function for each outcome that takes a numpy array with results for the outcome of interest, and returns the robustness score**

In [1]:
import functools

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

def maxp(data):
    return np.sum(data<=0.75)/data.shape[0]
    
SMALLER = 'SMALLER'
LARGER = 'LARGER'

maxp = functools.partial(robustness, SMALLER, 0.75)
inertia = functools.partial(robustness, LARGER, 0.6)
reliability = functools.partial(robustness, LARGER, 0.99)
utility = functools.partial(robustness, LARGER, 0.75)

**2) Generate 4 random release policies, and evaluate them over 500 scenarios. Sample the scenarios using Monte Carlo sampling. Next evaulate your robustness function for 1, 2, 3, ... 500 scenarios for each outcome and visualize this. What can you tell about the convergernce of the robusntess metric as a function of the number of scenarios?**

## Importing the Model

In [ ]:
from Week_5_6_quaq.dps_lake_model import lake_model as lake_problem

from ema_workbench import (Model, RealParameter, ScalarOutcome, Constant, SequentialEvaluator, MultiprocessingEvaluator,
                           ema_logging, Constraint, Policy, save_results)
from ema_workbench.analysis import (parcoords, prim)
import matplotlib.pyplot as plt

ema_logging.log_to_stderr(ema_logging.INFO)

lake_model = Model('lakemodel', function=lake_problem)
lake_model.time_horizon = 100

lake_model.uncertainties = [RealParameter("mean", 0.01, 0.05),
                            RealParameter("stdev", 0.001, 0.005),
                            RealParameter("b", 0.1, 0.45),
                            RealParameter("q", 2, 4.5),
                            RealParameter("delta", 0.93, 0.99)]

lake_model.levers = [RealParameter("c1", -2, 2),
                     RealParameter("c2", -2, 2),
                     RealParameter("r1", 0, 2),
                     RealParameter("r2", 0, 2),
                     RealParameter("w1", 0, 1)]

lake_model.outcomes = [ScalarOutcome("max_P", kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome("utility", kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome("inertia", kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome("reliability", kind=ScalarOutcome.MAXIMIZE)]

lake_model.constants = [Constant("alpha", 0.4),
                        Constant("nsamples", 100),
                        Constant("myears", 100)]
n_scenario = 500

In [4]:
import random
from ema_workbench import Policy

policies = []
for i in range(4):
    policy_dict = {"c1": random.randint(-2, 2),
                   "c2": random.randint(-2, 2),
                   "r1": random.randint(0, 2),
                   "r2": random.randint(0, 2),
                   "w1": random.randint(0, 1)}
    policies.append(Policy(f"pol_{i}", **policy_dict))

In [ ]:
monte_sample = samplers.MonteCarloSampler()
samples = monte_sample.sample(distribution='integer', params=(0, 500), size=100)

In [ ]:
for scenarios in range(1, 500):
    with MultiprocessingEvaluator(lake_model) as evaluator:
        results, outcomes = evaluator.perform_experiments(scenarios, policies=policies)

    max_p_outcomes = outcomes['max_P']
    utility_outcomes = outcomes['utility']
    inertia_outcomes = outcomes['inertia']
    reliability_outcomes = outcomes['reliability']
    max_p_robustness.append(robustness(SMALLER, 0.75, max_p_outcomes))
    utility_robustness.append(robustness(LARGER, 0.75, utility_outcomes))
    inertia_robustness.append(robustness(LARGER, 0.6, inertia_outcomes))
    reliability_robustness.append(robustness(LARGER, 0.99, reliability_outcomes))

In [ ]:
plt.plot(max_p_robustness)
plt.plot(inertia_robustness)
plt.plot(utility_robustness)
plt.plot(reliability_robustness)
plt.show()

After having run the model, there were several observation one could make. First of all, the reliability robustness scores tends to converge towards around 20% and the utility around 60%. Moreover, the inertia is (almost) always at 1 and the max pollution follows the reliability robustness score

## Searching for candidate solutions
Set up the robust optimization problem using the robustness functions you have specified. Assume that you will need 50 scenarios for estimating the robustness. Use $\epsilon$-progress and hypervolume to track convergence. Solve the optimization problem. As $\epsilon$ values, you can assume 0.05 for each of the four robustness metrics.

*note: this optimization problem is computationally very expensive. Develop and test your code using a sequential evaluator, a low number of function evaluations (e.g., 200), and a low number of scenarios (e.g., 5). Once everything seems to be working replace the sequential evaluator with an multiprocessing or ipyparallel evaluator, and increase the number of nfe and scenarios*.


In [ ]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    archive, convergence = evaluator.robust_optimize(robustness_function, 
                                                     scenarios=n_scenario, 
                                                     policies=policies, 
                                                     nfe=10,
                                                     convergence=convergence_metrics, 
                                                     epsilons=[0.05, 0.05, 0.05, 0.05])

In [ ]:
output = archive.iloc[:, 100 : :]
limits = parcoords.get_limits(output)
limits.loc[0, :] = 0
limits.loc[1, :] = 1

axes = parcoords.ParallelAxes(limits)
axes.plot(output)
plt.show()

**Plot your $\epsilon$-progress to evaluate convergergence, and visualize the trade-offs using parallel coordinate plots**

In [33]:
fig, axes = plt.subplots(ncols=2, sharex=True)
axes[0].plot(convergence.nfe, convergence.epsilon_progress)
axes[1].plot(convergence.nfe, convergence.hypervolume)

axes[0].set_xlabel('# nfe')
axes[1].set_xlabel('# nfe')

sns.despine()

plt.show()

Unfortunately, it was not possible for our group to run this model on time. However, we expect that convergence will happen only after a significant number of nfes (likely more than 100,000).

**What does this plot tell us about the tradeoffs and conflicting objectives?**

## Re-evaluate candidate solutions under uncertainty

We have used only 50 scenarios for the optimization. Take the results and re-evaluate them over a larger set (assume 1000 scenarios). How different are your results? What does this imply for the assumption of 50 scenarios during robust optimization.

*hint: use the to_dict method on a dataframe, next generate Policy objects in a list expression by iterating over the dicts returned by the to_dict method*

As we do not have the previous results, it is not possible to conduct this part of the code. However, the code itself will be similar to some of the previous assignments, where different policies are again created from outcomes.

## Comparison
If you have time, import your solutions found for MORDM and re-evaluate them over the same set of scnearios as used for re-evaluating the MORO results. Compare the robustness of MORDM and MORO, what do you observe?